In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
%matplotlib inline

In [2]:
# tsvファイルを読み込む
train_df = pd.read_csv('train.tsv',delimiter='\t')

In [3]:
train_df.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [7]:
len(train_df)

1482535

In [4]:
# 欠損値の確認
train_df.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          4
dtype: int64

In [5]:
def isBrandname(x,name):
    brand_name = str(x['brand_name'])
    if(brand_name==name):
        return 0
    return 1

In [6]:
fill_nan_brand_name = "Unknown"
train_df['brand_name']=train_df[['brand_name']].fillna(fill_nan_brand_name)
train_df['is_brand_name'] =  train_df.apply(lambda x: isBrandname(x,fill_nan_brand_name), axis=1)

In [7]:
train_df.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,is_brand_name
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,Unknown,10.0,1,No description yet,0
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,1
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,1
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,Unknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,0
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,Unknown,44.0,0,Complete with certificate of authenticity,0


In [8]:
# item_condition_idの割合をみる
train_df['item_condition_id'].value_counts()

1    640549
3    432161
2    375479
4     31962
5      2384
Name: item_condition_id, dtype: int64

In [9]:
# category_nameの割合をみる
train_df['category_name'].value_counts()

Women/Athletic Apparel/Pants, Tights, Leggings                 60177
Women/Tops & Blouses/T-Shirts                                  46380
Beauty/Makeup/Face                                             34335
Beauty/Makeup/Lips                                             29910
Electronics/Video Games & Consoles/Games                       26557
Beauty/Makeup/Eyes                                             25215
Electronics/Cell Phones & Accessories/Cases, Covers & Skins    24676
Women/Underwear/Bras                                           21274
Women/Tops & Blouses/Blouse                                    20284
Women/Tops & Blouses/Tank, Cami                                20284
Women/Dresses/Above Knee, Mini                                 20082
Women/Jewelry/Necklaces                                        19758
Women/Athletic Apparel/Shorts                                  19528
Beauty/Makeup/Makeup Palettes                                  19103
Women/Shoes/Boots                 

In [10]:
def get_first_category_name(x):
    category_name = str(x['category_name'])
    return category_name.split('/')[0]

In [11]:
# category_nameの第一カテゴリを抜き取る
# 欠損値は Unknown で埋めておく
fill_nan_category_name = "Unknown"
train_df['category_name']=train_df[['category_name']].fillna(fill_nan_category_name)

train_df['first_category_name'] =  train_df.apply(lambda x: get_first_category_name(x), axis=1)

In [13]:
train_df['first_category_name'].value_counts()

Women                     664385
Beauty                    207828
Kids                      171689
Electronics               122690
Men                        93680
Home                       67871
Vintage & Collectibles     46530
Other                      45351
Handmade                   30842
Sports & Outdoors          25342
Unknown                     6327
Name: first_category_name, dtype: int64

In [14]:
def get_second_category_name(x):
    category_name = str(x['category_name'])
    if(len(category_name.split('/')) > 1):
        return category_name.split('/')[1]
    else:
        return "NoCategory"

In [15]:
# category_nameの第二カテゴリを抜き取る
train_df['second_category_name'] =  train_df.apply(lambda x: get_second_category_name(x), axis=1)

In [16]:
train_df['second_category_name'].value_counts()

Athletic Apparel             134383
Makeup                       124624
Tops & Blouses               106960
Shoes                        100452
Jewelry                       61763
Toys                          58158
Cell Phones & Accessories     53290
Women's Handbags              45862
Dresses                       45758
Women's Accessories           42350
Jeans                         36130
Video Games & Consoles        35971
Sweaters                      35752
Underwear                     33784
Skin Care                     29838
Home Décor                    25188
Fragrance                     24294
Kitchen & Dining              21954
Tops                          20591
Other                         20017
Swimwear                      18924
Men's Accessories             18672
Girls 2T-5T                   18448
Coats & Jackets               18391
Girls 0-24 Mos                17878
Boys 0-24 Mos                 15372
Girls (4+)                    15243
Boys (4+)                   

In [17]:
def get_third_category_name(x):
    category_name = str(x['category_name'])
    if(len(category_name.split('/')) > 2):
        return category_name.split('/')[2]
    else:
        return "NoCategory"

In [18]:
# category_nameの第三カテゴリを抜き取る
train_df['third_category_name'] =  train_df.apply(lambda x: get_third_category_name(x), axis=1)

In [19]:
train_df['third_category_name'].value_counts()

Pants, Tights, Leggings       60177
Other                         50224
Face                          50171
T-Shirts                      46380
Shoes                         32168
Games                         30906
Lips                          30871
Athletic                      27059
Eyes                          26038
Cases, Covers & Skins         24676
Shorts                        21815
Bras                          21274
Blouse                        20288
Tank, Cami                    20284
Boots                         20270
Above Knee, Mini              20082
Necklaces                     19758
Makeup Palettes               19103
Women                         18975
Shirts & Tops                 16876
Sandals                       15368
Shoulder Bag                  15328
Fashion Sneakers              15203
T-shirts                      15108
Wallets                       14884
Knee-Length                   14770
Tops & T-Shirts               14508
Bracelets                   

In [20]:
train_df.sort_values(by=["price"], ascending=False).head(10)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,is_brand_name,first_category_name,second_category_name,third_category_name
760469,760469,NEW Chanel WOC Caviar Gold Hardware,1,Women/Women's Handbags/Shoulder Bag,Chanel,2009.0,0,"New with tag, box and dustbag but no receipt. ...",1,Women,Women's Handbags,Shoulder Bag
1262245,1262245,NEW-Chanel Boy Wallet o Chain WOC Caviar,1,Women/Women's Handbags/Messenger & Crossbody,Chanel,2006.0,0,Brand: Chanel Style: WOC Boy Wallet on Chain T...,1,Women,Women's Handbags,Messenger & Crossbody
1393600,1393600,David Yurman Wheaton ring,2,Women/Jewelry/Rings,David Yurman,2004.0,0,David Yurman Wheaton wing. Size 6. Original re...,1,Women,Jewelry,Rings
742113,742113,Chanel Chevron Fuschia Pink 2,1,Women/Women's Handbags/Shoulder Bag,Chanel,2000.0,1,Authentic Brand new The price for this is [rm]...,1,Women,Women's Handbags,Shoulder Bag
956343,956343,Chanel Black wallet on chain,1,Women/Women's Handbags/Messenger & Crossbody,Chanel,2000.0,1,Lambskin Color is Black with dark Silver Hardw...,1,Women,Women's Handbags,Messenger & Crossbody
1445348,1445348,Vs2/I 1ct Diamond Engagement Ring,2,Women/Jewelry/Rings,Unknown,2000.0,1,Absolutely stunning diamond engagement ring Pu...,0,Women,Jewelry,Rings
778940,778940,Mary kay,1,Beauty/Makeup/Face,Mary Kay,2000.0,1,30 time wise sets Oily to combo skin,1,Beauty,Makeup,Face
415027,415027,Chanel Classic Jumbo Single flap bag,3,Women/Women's Handbags/Shoulder Bag,Chanel,2000.0,1,Authentic. Pre-loved in Excellent Condition. P...,1,Women,Women's Handbags,Shoulder Bag
1250053,1250053,Brand New Chanel CC Quilted WOC,2,Women/Women's Handbags/Messenger & Crossbody,Chanel,2000.0,1,This is a brand new Chanel CC Quilted Wallet o...,1,Women,Women's Handbags,Messenger & Crossbody
952000,952000,Chanel J12 H0682 Wrist Watch for Women,3,Women/Women's Accessories/Watches,Unknown,1999.0,1,"HELLO , I HAVE A BLACK CHANEL WATCH J12 CERAMI...",0,Women,Women's Accessories,Watches


In [12]:
brand_dict = dict(train_df['brand_name'].value_counts())

In [13]:
def extract_brandname_from_name(x,b_dict):
    name = str(x['name'])
    name_split_list = name.split(' ')
    if(x['is_brand_name'] != 1):
        for item in name_split_list:
            if(item in b_dict):
                return item
    return str(x['brand_name'])

In [14]:
train_df['new_brand_name'] =  train_df.apply(lambda x: extract_brandname_from_name(x,brand_dict), axis=1)

In [37]:
# is_brand_nameをnew_brand_nameに対しても同じことを行い新しいものを作成する
def is_new_brand_name(x,name):
    brand_name = str(x['new_brand_name'])
    if(brand_name==name):
        return 0
    return 1

In [38]:
train_df['is_new_brand_name'] =  train_df.apply(lambda x: is_new_brand_name(x,fill_nan_brand_name), axis=1)

In [39]:
train_df['is_brand_name'].value_counts()

1    849853
0    632682
Name: is_brand_name, dtype: int64

In [40]:
train_df['is_new_brand_name'].value_counts()

1    978176
0    504359
Name: is_new_brand_name, dtype: int64

In [43]:
# new_brand_nameがUnknownでは無いのもので、そのカテゴリ数が多いもの = メジャーなものかどうか区別する
def is_major_brand_name(x,b_dict):
    search_name = str(x['new_brand_name'])
    result = 0
    if(search_name != "Unknown"):
        result = b_dict[search_name]
        if(result > 150):
            return 1
    return 0

In [44]:
train_df['is_major_brand'] =  train_df.apply(lambda x: is_major_brand_name(x,brand_dict), axis=1)

In [45]:
train_df['is_major_brand'].value_counts()

1    844138
0    638397
Name: is_major_brand, dtype: int64

In [47]:
train_df.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,is_brand_name,first_category_name,second_category_name,third_category_name,new_brand_name,is_new_brand_name,is_major_brand
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,Unknown,10.0,1,No description yet,0,Men,Tops,T-shirts,MLB,1,1
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,1,Electronics,Computers & Tablets,Components & Parts,Razer,1,0
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,1,Women,Tops & Blouses,Blouse,Target,1,1
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,Unknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,0,Home,Home Décor,Home Décor Accents,Unknown,0,0
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,Unknown,44.0,0,Complete with certificate of authenticity,0,Women,Jewelry,Necklaces,Unknown,0,0


In [48]:
train_df.to_csv('new_train.tsv',sep='\t',index=False)